In [1]:
!pip install albumentations


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import cv2
import albumentations as A
import xml.etree.ElementTree as ET
import os

# Define the augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.1, -0.1), contrast_limit=0, p=0.3),  # Low brightness
    A.Blur(blur_limit=5, p=0.1),
    A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.3),  # controls all color-related params
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids'], min_visibility=0.1))

# Parse Pascal VOC XML file
def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    bboxes = []
    labels = []

    for obj in root.findall("object"):
        label = obj.find("name").text
        bbox = obj.find("bndbox")
        xmin, ymin, xmax, ymax = map(int, [
            bbox.find("xmin").text, bbox.find("ymin").text,
            bbox.find("xmax").text, bbox.find("ymax").text
        ])
        bboxes.append([xmin, ymin, xmax, ymax])
        labels.append(label)

    return bboxes, labels, tree, root

# Update XML file with new bounding boxes and image size
def update_xml(xml_path, new_bboxes, new_labels, tree, root, new_filename, new_shape):
    root.find("filename").text = new_filename
    root.find("size").find("width").text = str(new_shape[1])
    root.find("size").find("height").text = str(new_shape[0])
    root.find("size").find("depth").text = str(new_shape[2])

    objects = root.findall("object")
    for i, obj in enumerate(objects):
        if i >= len(new_bboxes):  # Avoid index error
            root.remove(obj)
            continue
        bbox = obj.find("bndbox")
        bbox.find("xmin").text = str(int(new_bboxes[i][0]))
        bbox.find("ymin").text = str(int(new_bboxes[i][1]))
        bbox.find("xmax").text = str(int(new_bboxes[i][2]))
        bbox.find("ymax").text = str(int(new_bboxes[i][3]))
        obj.find("name").text = new_labels[i]

    tree.write(xml_path)

# Augment the dataset in a folder
def augment_dataset(input_folder, output_folder, num_aug=15):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png'))]

    for file in image_files:
        image_path = os.path.join(input_folder, file)
        xml_path = os.path.splitext(image_path)[0] + '.xml'

        if not os.path.exists(xml_path):
            print(f"⚠️ XML not found for {file}, skipping.")
            continue

        image = cv2.imread(image_path)
        bboxes, labels, xml_tree, xml_root = parse_xml(xml_path)

        count = 0
        attempts = 0
        while count < num_aug and attempts < num_aug * 3:  # Allow retries in case of bbox loss
            augmented = transform(image=image, bboxes=bboxes, category_ids=labels)
            aug_image = augmented["image"]
            aug_bboxes = augmented["bboxes"]
            aug_labels = augmented["category_ids"]

            # Skip if any bbox is lost
            if len(aug_bboxes) != len(bboxes):
                attempts += 1
                continue

            base_name = os.path.splitext(file)[0]
            new_image_name = f"{base_name}_aug_{count+1}.jpg"
            new_image_path = os.path.join(output_folder, new_image_name)
            new_xml_path = os.path.join(output_folder, f"{base_name}_aug_{count+1}.xml")

            cv2.imwrite(new_image_path, aug_image)
            update_xml(new_xml_path, aug_bboxes, aug_labels, xml_tree, xml_root, new_image_name, aug_image.shape)
            count += 1
            attempts += 1

        print(f"✅ {file}: {count} augmentations saved.")

# === Configure your folders ===
input_folder = r"C:\\MTP1\\augmented\\Original without aug data"      # Folder with original .jpg and .xml files
output_folder = r"C:\\MTP1\\augmented\\aug1"         # Folder to save augmented files

# === Run the augmentation ===
augment_dataset(input_folder, output_folder, num_aug=15)


✅ duck (1).png: 15 augmentations saved.
✅ duck (10).png: 15 augmentations saved.
✅ duck (11).png: 15 augmentations saved.
✅ duck (12).png: 15 augmentations saved.
✅ duck (13).png: 15 augmentations saved.
✅ duck (14).png: 15 augmentations saved.
✅ duck (16).png: 15 augmentations saved.
✅ duck (17).png: 15 augmentations saved.
✅ duck (19).png: 15 augmentations saved.
✅ duck (2).png: 15 augmentations saved.
✅ duck (20).png: 15 augmentations saved.
✅ duck (21).png: 15 augmentations saved.
✅ duck (22).png: 15 augmentations saved.
✅ duck (23).png: 15 augmentations saved.
✅ duck (24).png: 15 augmentations saved.
✅ duck (25).png: 15 augmentations saved.
✅ duck (26).png: 15 augmentations saved.
✅ duck (27).png: 15 augmentations saved.
✅ duck (28).png: 15 augmentations saved.
✅ duck (29).png: 15 augmentations saved.
✅ duck (3).png: 15 augmentations saved.
✅ duck (30).png: 15 augmentations saved.
✅ duck (31).png: 15 augmentations saved.
✅ duck (32).png: 15 augmentations saved.
✅ duck (33).png: 15